 
# Federated Learning Training Plan: Execute Plan

Here we load and execute Plan and Model params created earlier in "Create Plan" notebook. 

In [1]:
%load_ext autoreload
%autoreload 2
import syft as sy
import torch as th
from torchvision import datasets, transforms
from syft.serde import protobuf
import base64
from syft_proto.messaging.v1.plan_pb2 import Plan as PlanPB
from syft_proto.messaging.v1.state_pb2 import State as StatePB
from syft_proto.types.torch.v1.script_function_pb2 import ScriptFunction as ScriptFunctionPB
import os
import numpy as np

sy.hook(globals())

C:\Users\Vova\AppData\Local\conda\conda\envs\pysyft\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Vova\AppData\Local\conda\conda\envs\pysyft\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Vova\AppData\Local\conda\conda\envs\pysyft\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Vova\AppData\Loc

Setting up Sandbox...
Done!


Utility func that unserializes file contents into PySyft classes.
Note that we must know file contents beforehand to use specific protobuf class for deserialization.

In [2]:
def deserializeFromBase64(worker, filename, pb):
    
    with open(filename, "r") as f:
        str = f.read()
    bin = base64.b64decode(str)
    pb.ParseFromString(bin)
    return protobuf.serde._unbufferize(worker, pb)

## Step 5: Unserialize 

In [3]:
training_plan_ops = deserializeFromBase64(hook.local_worker, "tp_ops.b64", PlanPB())
training_plan_ts = deserializeFromBase64(hook.local_worker, "tp_ts.b64", ScriptFunctionPB())
model_params_state = deserializeFromBase64(hook.local_worker, "model_params.b64", StatePB())
# unwrap tensors from State
model_params = model_params_state.tensors()

print("Loaded plan (# of ops):", len(training_plan_ops.operations))
print("Loaded tracescript plan code:", training_plan_ts.code)
print("Loaded params count:", len(model_params))

Loaded plan (# of ops): 43
Loaded tracescript plan code: def forward(self,
    argument_1: Tensor,
    argument_2: Tensor,
    argument_3: Tensor,
    argument_4: Tensor,
    argument_5: Tensor,
    argument_6: Tensor,
    argument_7: Tensor,
    argument_8: Tensor) -> Tuple[Tensor, Tensor, Tensor, Tensor, Tensor, Tensor]:
  _0 = torch.matmul(argument_1, torch.t(argument_5))
  _1 = torch.add(_0, argument_6, alpha=1)
  _2 = torch.relu(_1)
  _3 = torch.add(torch.matmul(_2, torch.t(argument_7)), argument_8, alpha=1)
  _4 = torch.softmax(_3, 1, None)
  _5 = torch.mean(torch.mul(argument_2, torch.log(_4)), dtype=None)
  _6 = torch.neg(_5)
  _7 = ops.prim.NumToTensor(torch.size(argument_2, 1))
  _8 = torch.div(torch.sub(_4, argument_2, alpha=1), torch.mul(argument_3, _7))
  _9 = torch.matmul(_8, argument_7)
  _10 = torch.to(torch.gt(_1, 0), 6, False, False, None)
  _11 = torch.mul(_9, _10)
  _12 = torch.matmul(torch.t(_11), argument_1)
  _13 = torch.sum(_11, [0], False, dtype=None)
  _14 = t

## Step 6: Train!

In [4]:
batch_size = 64
mnist = th.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True
)

def execute_training_plan(data, plan, model_params, epochs=3, batch_size=th.tensor(batch_size), lr=th.tensor(0.01)):
    for epoch in range(1, epochs+1):
        losses = []
        accuracies = []
        for batch_idx, (X, y) in enumerate(data):
            X = X.view(X.shape[0], -1)
            y_oh = th.nn.functional.one_hot(y, 10)
            *model_params, loss, acc = plan(X, y_oh, batch_size, lr, *model_params)
            losses.append(loss.item())
            accuracies.append(acc.item())
            if batch_idx % 100 == 0:
                print("Batch %d, loss: %f, accuracy: %f" % (batch_idx, loss, acc), end="\r")
        print('Epoch %d, avg loss: %f, avg training accuracy: %f' % (epoch, np.mean(losses), np.mean(accuracies)))

In [5]:
# Plain Plan
execute_training_plan(mnist, training_plan_ops, model_params)

Epoch 1, avg loss: 0.207533, avg training accuracy: 0.444013
Epoch 2, avg loss: 0.157925, avg training accuracy: 0.732126
Epoch 3, avg loss: 0.119505, avg training accuracy: 0.789346


In [6]:
# Torchscript Plan
execute_training_plan(mnist, training_plan_ts, model_params)

Epoch 1, avg loss: 0.094847, avg training accuracy: 0.818563
Epoch 2, avg loss: 0.079739, avg training accuracy: 0.835871
Epoch 3, avg loss: 0.069991, avg training accuracy: 0.846848
